# <center>Welcome FSP solved with local search based metaheuristics</center>

This notebook presents a practical approach to solving the flowshop problem by implementing well-known local search based metaheuristics. These metaheuristics are effective in generating high-quality solutions for large instances of the problem, requiring only a reasonable amount of computational resources. Compared to heuristics, local search metaheuristics are more effective for solving the flowshop problem because they can escape from local optima and find better solutions

# Table of Contents

1. [Johnson **n** jobs **2** machines](#Johnson-**n**-jobs-**2**-machines)
2. [CDS Heuristic](#cds-heuristic)
3. [NEH Heuristic](#neh-heuristic)
4. [Ham Heuristic](#ham-heuristic)
5. [Palmer Heuristic](#palmer-heuristic)
6. [PRSKE Heuristic](#prske-heuristic)
8. [Chen Heuristic](#Chen-heuristic-(1983))
7. [Weighted CDS Heuristic](#Weighted-CDS-heuristic)
9. [Gupta Heuristic](#gupta-heuristic)
10. [NRH Heuristic (NEW Ramzi Heuristic)](#nrh-heuristic-new-ramzi-heuristic)
11. [Kusiak Heuristic ](#Kusiak-heuristic)
12. [ Comparison Figure   ](#Comparison-Figure)




## Data utils

In [1]:
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks, upper_bound

### Path Cost calculation function :
Used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [2]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]

    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

### Gantt graph generator

In [3]:
def generate_gantt_chart(processing_times, seq, interval=50, labeled=True):
    data = processing_times.T
    nb_jobs, nb_machines = processing_times.shape
    schedules = np.zeros((nb_machines, nb_jobs), dtype=dict)
    # schedule first job alone first
    task = {"name": "job_{}".format(
        seq[0]+1), "start_time": 0, "end_time": data[0][seq[0]]}

    schedules[0][0] = task
    for m_id in range(1, nb_machines):
        start_t = schedules[m_id-1][0]["end_time"]
        end_t = start_t + data[m_id][0]
        task = {"name": "job_{}".format(
            seq[0]+1), "start_time": start_t, "end_time": end_t}
        schedules[m_id][0] = task

    for index, job_id in enumerate(seq[1::]):
        start_t = schedules[0][index]["end_time"]
        end_t = start_t + data[0][job_id]
        task = {"name": "job_{}".format(
            job_id+1), "start_time": start_t, "end_time": end_t}
        schedules[0][index+1] = task
        for m_id in range(1, nb_machines):
            start_t = max(schedules[m_id][index]["end_time"],
                          schedules[m_id-1][index+1]["end_time"])
            end_t = start_t + data[m_id][job_id]
            task = {"name": "job_{}".format(
                job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][index+1] = task

    # create a new figure
    fig, ax = plt.subplots(figsize=(18, 8))

    # set y-axis ticks and labels
    y_ticks = list(range(len(schedules)))
    y_labels = [f'Machine {i+1}' for i in y_ticks]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    # calculate the total time
    total_time = max([job['end_time'] for proc in schedules for job in proc])

    # set x-axis limits and ticks
    ax.set_xlim(0, total_time)
    x_ticks = list(range(0, total_time+1, interval))
    ax.set_xticks(x_ticks)

    # set grid lines
    ax.grid(True, axis='x', linestyle='--')

    # create a color dictionary to map each job to a color
    color_dict = {}
    for proc in schedules:
        for job in proc:
            if job['name'] not in color_dict:
                color_dict[job['name']] = (np.random.uniform(
                    0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1))

    # plot the bars for each job on each processor
    for i, proc in enumerate(schedules):
        for job in proc:
            start = job['start_time']
            end = job['end_time']
            duration = end - start
            color = color_dict[job['name']]
            ax.barh(i, duration, left=start, height=0.5,
                    align='center', color=color, alpha=0.8)
            if labeled:
                # add job labels
                label_x = start + duration/2
                label_y = i
                ax.text(
                    label_x, label_y, job['name'][4:], ha='center', va='center', fontsize=10)

    plt.show()

## Neighborhood generation

### SWAP

In [9]:
def swap(solution, i, k):
    solution[i], solution[k] = solution[k], solution[i]
    return solution

### Random SWAP

In [5]:
def random_swap(solution):
    i = random.choice(list(solution))
    j = random.choice(list(solution))

    new_solution = swap(solution, i, j)
    
    return new_solution

### Best SWAP

In [6]:
def best_swap(solution, i):
    pass 

### First Admissible SWAP

In [7]:
def first_admissible_swap(solution, i):
    pass 

### First and Best Admissible SWAP

In [8]:
def fba_swap(solution, i):
    pass 

## Local Based Metaheuristics

### Simulated annealing (RS) 

## Tests